# image manipulation in python
a notebook to learn a few basics about image manipulation

In [ ]:
pip install matplotlib

In [ ]:
pip install numpy==1.26.3

In [ ]:
pip install aicsimageio==4.14.0

In [ ]:
pip install aicsimageio[nd2]

In [ ]:
pip install scikit-image

In [ ]:
pip install numpy

next we will load the packages that we need

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from aicsimageio import AICSImage
from skimage.filters import gaussian

## load and display an image
next we will load an image, i.e. an nd2 file (Nikon's file format)

In [ ]:
# load an nd2 file
img = AICSImage('../data/nd2/WT_001.nd2')
data = img.get_image_data("TCZYX")  # Choose the correct dimension order
data.shape

after that let's display the image to see what we are working with

In [ ]:
# Pick the first timepoint and z-slice
t_index = 0
z_index = 0

# Extract each channel slice (assume 3 channels, for example)
channel_0 = data[t_index, 0, z_index, :, :]
channel_1 = data[t_index, 1, z_index, :, :]
channel_2 = data[t_index, 2, z_index, :, :]

# make a quick figure to display individual channels
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(channel_0, cmap='gray')
axes[0].set_title("Channel 0")

axes[1].imshow(channel_1, cmap='gray')
axes[1].set_title("Channel 1")

axes[2].imshow(channel_2, cmap='gray')
axes[2].set_title("Channel 2")

plt.tight_layout()
plt.show()

as you can see x- and y- coordinates are displayed in pixel numbers, we can write a function to retrieve the actual dimensions from the image's metadata

In [ ]:
# extract meta data, i.e. size of voxel in µm
def get_voxel_size_from_aics_image(aics_image):
    return (aics_image.physical_pixel_sizes.Z,
            aics_image.physical_pixel_sizes.Y,
            aics_image.physical_pixel_sizes.X)

we can call the function by its name and run input an image

In [ ]:
voxel = get_voxel_size_from_aics_image(img)

voxel

since we are working with a single image plane, z has no meaning and each pixel is 0.176.. µm

we can use these numbers now to calculate the dimensions of our image

In [ ]:
x_dimension = data.shape[4] * voxel[1]
y_dimension = data.shape[3] * voxel[1]

print(f"length in x: {x_dimension:.2f} µm")
print(f"length in y: {y_dimension:.2f} µm")

## basic image transformations
let's learn some basic image transformations, we will be using image filters scikit-image, to start with, we will be working with a single channel for simplicity

In [ ]:
gphn = channel_0
gphn.shape

In [ ]:
plt.imshow(gphn, cmap='Greys')

In [ ]:
plt.hist(gphn.ravel(), bins=4096)
plt.show()

next we will use a Gaussian filter and display the resulting image in order to smooth the image

In [ ]:
high = gaussian(gphn, sigma=10, preserve_range=True)
plt.imshow(high, cmap='Greys')

in this smooth image, gephyrin clusters cannot be seen, finally we will apply two gaussian filters (separately) and then subtract these images (pixel by pixel) from one another, this is called difference of Gaussians (DoG) and this method can be used to enhance certain features of an image

In [ ]:
plt.hist(high.ravel(), bins=4096)
plt.show()

In [ ]:
def sub_to_zero(a, b):
    # Element-wise subtraction and maximum with zero
    return np.maximum(a - b, 0)


# Process Gphn channel (Difference of Gaussian)
low = gaussian(gphn, sigma=2, preserve_range=True)
high = gaussian(gphn, sigma=10, preserve_range=True)

dog = sub_to_zero(low,high)

# make a quick figure for display
fig, axes = plt.subplots(1, 4, figsize=(12, 4))

axes[0].imshow(gphn, cmap='gray')
axes[0].set_title("original")

axes[1].imshow(low, cmap='gray')
axes[1].set_title("low gaussian")

axes[2].imshow(high, cmap='gray')
axes[2].set_title("high gaussian")

axes[3].imshow(dog, cmap='gray')
axes[3].set_title("difference")

plt.tight_layout()
plt.show()

In [ ]:
# make a quick figure to display individual channels
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(gphn, cmap='Grays')
axes[0].set_title("original")

axes[1].imshow(dog, cmap='Grays')
axes[1].set_title("difference")

plt.tight_layout()
plt.show()
